In [1]:
import pandas as pd
from datetime import datetime

# Read data from the Excel file
meter_list = pd.read_excel('gorilla_test_data.xlsx', sheet_name='meter_list')
forecast_data = pd.read_excel('gorilla_test_data.xlsx', sheet_name='forecast_table')
rate_data = pd.read_excel('gorilla_test_data.xlsx', sheet_name='rate_table')

# Convert dtype to datetime64 and coerce errors
forecast_data['date'] = pd.to_datetime(forecast_data['date'], errors='coerce', infer_datetime_format=True)
rate_data['date'] = pd.to_datetime(rate_data['date'], errors='coerce', infer_datetime_format=True)
forecast_data.head()
rate_data.head()

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524


In [2]:
# Meter_list functions 

def return_aq_kwh(df_list, meter_id): 
    if meter_id not in df_list['meter_id'].values:
        raise ValueError(f"Meter ID {meter_id} not found in the meter_list.")
    
    aq_kwh = df_list.loc[df_list['meter_id'] == meter_id, 'aq_kwh'].values[0]
    print(f"The meter with ID {meter_id} corresponds to this aq_kwh: {aq_kwh}")
    return aq_kwh

def return_exit_zone(df_list, meter_id): 
    if meter_id not in df_list['meter_id'].values:
        raise ValueError(f"Meter ID {meter_id} not found in the meter_list.")
    
    exit_zone = df_list.loc[df_list['meter_id'] == meter_id, 'exit_zone'].values[0]
    print(f"The meter with ID {meter_id} corresponds to this exit zone: {exit_zone}")
    return exit_zone

In [3]:
# Forecast data functions 

# Convert input date in the correct datetime format
def convert_input_date(input_date):
    try:
        # Try to convert using the "DD/MM/YYYY" format
        converted_input_date = pd.to_datetime(input_date, format='%d/%m/%Y')
        return converted_input_date
    except ValueError:
        try:
            # If the first attempt fails, try the "YYYY-MM-DD" format
            converted_input_date = pd.to_datetime(input_date, format='%Y-%m-%d')
            return converted_input_date
        except ValueError:
            # If both attempts fail, raise an error
            raise ValueError("Invalid date format. Please provide a valid date.")
    
def return_kwh(df_forecast, meter_id, input_date): 
    converted_input_date = convert_input_date(input_date)
    mask = (df_forecast['meter_id'] == meter_id) & (df_forecast['date'] == converted_input_date)
    kwh = df_forecast.loc[mask, 'kwh'].values[0]
    print(f'The kwh usage of {meter_id} on {converted_input_date.date()} is: {kwh}')
    return kwh


In [4]:
# Rate data functions 

# Function to find the closest past date
def find_closest_past_date(df_rate, input_date):
    input_date = convert_input_date(input_date)

    # Filter only past dates
    past_dates = df_rate[rate_data['date'] <= input_date]['date']

    # If there are past dates, find the closest one
    if not past_dates.empty:
        closest_past_date = past_dates.iloc[(past_dates - input_date).abs().idxmin()]
        print(f"The date from which on the rates are applicable is: {closest_past_date.date()}")
        return closest_past_date
    else:
        return None
    
# Create function to retrieve rate_p_per_kwh on date by meter_id and corresponding exit_zone. Return = array with 3 possible prices
def return_rate_by_date_meter_exit(df_list, meter_id, df_rate, input_date, exit_zone = None):
    converted_date = convert_input_date(input_date)
    rate_day = find_closest_past_date(df_rate, converted_date)
    if exit_zone == None: 
        exit_zone = return_exit_zone(df_list, meter_id)
    mask = (df_rate['exit_zone'] == exit_zone) & (df_rate['date'] == rate_day)
    rate_p_per_kwh = df_rate.loc[mask, ['aq_min_kwh', 'aq_max_kwh', 'rate_p_per_kwh']]
    return rate_p_per_kwh


In [19]:
# Test return_rate_by_date_meter_exit
test = return_rate_by_date_meter_exit(meter_list, 14676236, rate_data, "11/11/2023")
test.head()

The date from which on the rates are applicable is: 2023-10-01
The meter with ID 14676236 corresponds to this exit zone: EA1


,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
798,0,73200.0,0.5863
799,73200,732000.0,0.4377
800,732000,NaN,0.6356


In [6]:
# Daily charge function

def return_daily_charge_by_date_meter_exit(df_list, meter_id, df_forecast, df_rate, input_date):
    converted_date = convert_input_date(input_date)
    rate_p_per_kwh = return_rate_by_date_meter_exit(df_list, meter_id, df_rate, input_date)
    aq_kwh = return_aq_kwh(df_list, meter_id)
    daily_kwh = return_kwh(df_forecast, meter_id, input_date)

    if (aq_kwh < rate_p_per_kwh.iloc[0,1]) & (aq_kwh >= 0): 
        print(f"The rate per kwh for {converted_date.date()} is: {rate_p_per_kwh.iloc[0,2]} pennies")
        daily_charge_p = daily_kwh * rate_p_per_kwh.iloc[0,2]
    elif aq_kwh < rate_p_per_kwh.iloc[1,1]: 
        print(f"The rate per kwh for {converted_date.date()} is: {rate_p_per_kwh.iloc[1,2]} pennies")
        daily_charge_p = daily_kwh * rate_p_per_kwh.iloc[1,2]
    elif aq_kwh > rate_p_per_kwh.iloc[2,0]: 
        print(f"The rate per kwh for {converted_date.date()} is: {rate_p_per_kwh.iloc[2,2]} pennies")
        daily_charge_p = daily_kwh * rate_p_per_kwh.iloc[2,2]
    else: 
        raise ValueError
    daily_charge_pounds = daily_charge_p * 0.01
    print(f'The daily_charge is:\n{daily_charge_p.round(2)} pennies\n{daily_charge_pounds.round(4)} pounds')
    return daily_charge_p, daily_charge_pounds

In [20]:
# Test daily charge function
return_daily_charge_by_date_meter_exit(meter_list, 14676236, forecast_data, rate_data, "01/06/2020")

The date from which on the rates are applicable is: 2020-04-01
The meter with ID 14676236 corresponds to this exit zone: EA1
The meter with ID 14676236 corresponds to this aq_kwh: 28978
The kwh usage of 14676236 on 2020-06-01 is: 22.0707680767784
The rate per kwh for 2020-06-01 is: 0.2652 pennies
The daily_charge is:
5.85 pennies
0.0585 pounds


(5.853167693961631, 0.05853167693961631)

In [8]:
# Table filters

# Date range filter
def daterange_filter(df_forecast, df_rate, start_date="01/04/2020", end_date="2030-12-31"): 
    start_date_converted = convert_input_date(start_date)
    end_date_converted = convert_input_date(end_date)

        # Check if start_date is earlier than the available data
    if (start_date_converted < df_rate['date'].min()):
        raise ValueError("Start date is earlier than the available data.")

    # Check if start_date is later than end_date
    if start_date_converted > end_date_converted:
        raise ValueError("Start date is later than end date.")
    
    # Check if start_date is later than the latest date in the forecast table
    if start_date_converted > df_forecast['date'].max():
        raise ValueError("Start date is later than the latest date available in the forecast table.")

    # Filter df_forecast based on the date range
    df_forecast_filtered = df_forecast[(df_forecast['date'] >= start_date_converted) & (df_forecast['date'] <= end_date_converted)]

    # Filter df_rate based on the date range
    closest_past_date_rate = find_closest_past_date(df_rate, start_date_converted)
    df_rate_filtered = df_rate[(df_rate['date'] >= closest_past_date_rate) & (df_rate['date'] <= end_date_converted)]
    return df_forecast_filtered, df_rate_filtered

def meter_id_filter(df_list, meter_id, df_forecast, df_rate): 
    exit_zone = return_exit_zone(df_list, meter_id)
    df_rate_filtered = df_rate[(df_rate['exit_zone'] == exit_zone)]
    df_forecast_filtered = df_forecast[(df_forecast['meter_id'] == meter_id)]
    return df_forecast_filtered, df_rate_filtered

# aq_kwh filter
def aq_kwh_filter(df_rate, aq_kwh):
    if (aq_kwh < df_rate.iloc[0, 3]) and (aq_kwh >= 0):
        # Keep 1st row, delete 2 rows, repeat
        df_rate_filtered = df_rate.iloc[::3]
    elif aq_kwh < df_rate.iloc[1, 3]:
        # Delete 1st row, keep 2nd row, delete 2 rows, repeat
        df_rate_filtered = df_rate.iloc[1::3]
    elif aq_kwh > df_rate.iloc[2, 2]:
        # Delete 1st and 2nd row, keep 3rd row, delete 2 rows, repeat
        df_rate_filtered = df_rate.iloc[2::3]
    else:
        raise ValueError("Invalid aq_kwh value.")
    return df_rate_filtered

def total_filter(df_list, meter_id, df_forecast, df_rate, start_date="01/04/2020", end_date="2030-12-31"): 
    df_forecast_filtered, df_rate_filtered = daterange_filter(df_forecast, df_rate, start_date, end_date)
    df_forecast_filtered, df_rate_filtered = meter_id_filter(df_list, meter_id, df_forecast_filtered, df_rate_filtered)
    aq_kwh = return_aq_kwh(df_list, meter_id)
    df_rate_filtered = aq_kwh_filter(df_rate_filtered, aq_kwh)
    return df_forecast_filtered, df_rate_filtered

In [9]:
# Test table filters seperate
test_forecast, test_rate = daterange_filter(forecast_data, rate_data, "01/06/2020", "02/06/2050")
test_forecast, test_rate = meter_id_filter(meter_list, 14676236, test_forecast, test_rate)
test_rate.head(20)
test_rate = aq_kwh_filter(test_rate, 100)
test_rate.head(20)

The date from which on the rates are applicable is: 2020-04-01
The meter with ID 14676236 corresponds to this exit zone: EA1


,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
114,2020-10-01,EA1,0,73200.0,0.2970
228,2021-04-01,EA1,0,73200.0,0.3327
342,2021-10-01,EA1,0,73200.0,0.3726
456,2022-04-01,EA1,0,73200.0,0.4173
570,2022-10-01,EA1,0,73200.0,0.4674
684,2023-04-01,EA1,0,73200.0,0.5235
798,2023-10-01,EA1,0,73200.0,0.5863
912,2024-04-01,EA1,0,73200.0,0.6566
1026,2024-10-01,EA1,0,73200.0,0.7354


In [23]:
# Test table filters combo // Forecast table filtered
fc, rate = total_filter(meter_list, 50264822 , forecast_data, rate_data, "01/06/2020", "02/06/2050")
fc.head(20)

The date from which on the rates are applicable is: 2020-04-01
The meter with ID 50264822 corresponds to this exit zone: NT1
The meter with ID 50264822 corresponds to this aq_kwh: 265667


,meter_id,date,kwh
1706,50264822,2020-06-01,278.759084
1707,50264822,2020-06-02,311.431685
1708,50264822,2020-06-03,302.010415
1709,50264822,2020-06-04,336.318520
1710,50264822,2020-06-05,364.082279
1711,50264822,2020-06-06,330.252118
1712,50264822,2020-06-07,374.350744
1713,50264822,2020-06-08,366.614457
1714,50264822,2020-06-09,389.885276
1715,50264822,2020-06-10,339.406926


In [24]:
# Test table filters combo // Rate table filtered
rate.head(20)

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
55,2020-04-01,NT1,73200,732000.0,0.2181
169,2020-10-01,NT1,73200,732000.0,0.2443
283,2021-04-01,NT1,73200,732000.0,0.2736
397,2021-10-01,NT1,73200,732000.0,0.3064
511,2022-04-01,NT1,73200,732000.0,0.3432
625,2022-10-01,NT1,73200,732000.0,0.3844
739,2023-04-01,NT1,73200,732000.0,0.4305
853,2023-10-01,NT1,73200,732000.0,0.4821
967,2024-04-01,NT1,73200,732000.0,0.5400
1081,2024-10-01,NT1,73200,732000.0,0.6048


In [52]:
# Create new dataframe based on fc_filtered and rate_filtered

def results_meter_id(fc_filtered, rate_filtered): 
    # Merge with a left join
    df_results = pd.merge(fc_filtered, rate_filtered, how='left', on='date')
    # Drop unnecessary columns
    df_results.drop(['aq_min_kwh', 'aq_max_kwh'], axis=1, inplace=True)
    # Fill in the first row with the most recent rates
    df_results[['exit_zone', 'rate_p_per_kwh']] = df_results[['exit_zone', 'rate_p_per_kwh']].fillna(rate_filtered.iloc[0][['exit_zone', 'rate_p_per_kwh']])
    # Forward fill the missing values in rate_filtered columns
    df_results[['exit_zone', 'rate_p_per_kwh']] = df_results[['exit_zone', 'rate_p_per_kwh']].ffill()
    return df_results



In [63]:
# Test new results dataframe
df_result = results_meter_id(fc, rate)
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853 entries, 0 to 852
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   meter_id        853 non-null    int64         
 1   date            853 non-null    datetime64[ns]
 2   kwh             853 non-null    float64       
 3   exit_zone       853 non-null    object        
 4   rate_p_per_kwh  853 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 40.0+ KB


In [64]:
# Calculate daily charges in pennies and pounds
def calculate_daily_charge(df_results): 
    df_results['daily_charge_p'] = df_results['kwh'] * df_results['rate_p_per_kwh']
    df_results['daily_charge_pounds'] = df_results['daily_charge_p'] * 0.01

    df_results['daily_charge_p'] = df_results['daily_charge_p'].round(2)
    df_results['daily_charge_pounds'] = df_results['daily_charge_pounds'].round(4)
    return df_results

In [67]:
# Check results daily charges
df_results = calculate_daily_charge(df_result)
df_results.head()

,meter_id,date,kwh,exit_zone,rate_p_per_kwh,daily_charge_p,daily_charge_pounds
0,50264822,2020-06-01,278.759084,NT1,0.2181,60.80,0.6080
1,50264822,2020-06-02,311.431685,NT1,0.2181,67.92,0.6792
2,50264822,2020-06-03,302.010415,NT1,0.2181,65.87,0.6587
3,50264822,2020-06-04,336.318520,NT1,0.2181,73.35,0.7335
4,50264822,2020-06-05,364.082279,NT1,0.2181,79.41,0.7941


In [76]:
# Aggregated by meter_id
def results_aggregated(df_results): 
    sum_results = df_results.groupby('meter_id')[['kwh', 'daily_charge_p', 'daily_charge_pounds']].sum()
    return sum_results

In [80]:
sum = results_aggregated(df_results)
sum.head()

,kwh,daily_charge_p,daily_charge_pounds
meter_id,,,
50264822,265667.0,58105.51,581.0551
